In [38]:
import sys
import os

if "../" not in sys.path:
    sys.path.append("../")

In [39]:
# import glob
# import shutil

# # Get a list of all CSV files in the source directory
# csv_files = glob.glob("../datasets/*.csv")

# # Move each CSV file to the destination directory
# for file in csv_files:
#     shutil.move(file, "../results/")

In [45]:
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from collections import namedtuple, defaultdict
from typing import List, Tuple, Set


class LPJS:
    def __init__(
        self,
        tasks: List[List[int]],
        orders: List[List[Tuple[int, int]]],
        capacities: dict,
    ):
        self.tasks = tasks
        self.orders = orders
        self.capacities = capacities
        self.n_jobs = len(tasks)
        self.model = pywraplp.Solver(
            "JSSP", pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING
        )

    def addConstraints(self):
        # task = namedtuple("task", "start end interval")

        #         """
        #         Ràng buộc thời gian bắt đầu và kết thúc của mỗi task.
        #         Điều kiện :
        #             - Thời gian bắt đầu và kết thúc của mỗi task phải thỏa mãn điều kiện : 0 <= start <= end <= horizon
        #             -  start + duration = end
        #         """
        self.start_vars = [defaultdict() for _ in range(self.n_jobs)]
        self.end_vars = [defaultdict() for _ in range(self.n_jobs)]
        # define a dict of tasks including what machine can do the task
        all_tasks = set.union(*[set(i) for i in self.tasks])
        task_machines = defaultdict(dict)
        for taskJ in all_tasks:
            for machine in self.capacities:
                for taskM in self.capacities[machine]:
                    if taskJ == taskM:
                        task_machines[taskJ][machine] = self.capacities[machine][taskM]
                        break
        print(task_machines)
        # Define variables for assigning task of each job to each machine
        # assignment = defaultdict()
        assignment_task = defaultdict(list)
        for job in range(self.n_jobs):
            for task in self.tasks[job]:
                self.start_vars[job][task] = self.model.NumVar(
                    0, self.model.infinity(), f"start_job_{job}_task_{task}"
                )
                self.end_vars[job][task] = self.model.NumVar(
                    0, self.model.infinity(), f"end_job_{job}_task_{task}"
                )
                for machine in task_machines[task]:
                    decision_var = self.model.BoolVar(
                        f"job_{job}_task_{task}_machine_{machine}"
                    )
                    assignment_task[(job, task)].append(
                        (decision_var, machine, task_machines[task][machine][1])
                    )
                self.model.Add(
                    sum(des for des, _, _ in assignment_task[(job, task)]) == 1
                )  # each task must be assigned to one machine

        for job in range(self.n_jobs):
            for task in self.tasks[job]:
                for decision_var, machine, _ in assignment_task[(job, task)]:
                    self.model.Add(
                        self.start_vars[job][task]
                        + decision_var * task_machines[task][machine][0]
                        <= self.end_vars[job][task]
                    )
        # Orders constraint
        for job, order in enumerate(self.orders):
            for task1, task2 in order:
                self.model.Add(self.end_vars[job][task1] <= self.start_vars[job][task2])
        # Minimize cost
        self.model.Minimize(
            sum(
                cost * decision_var
                for job in range(self.n_jobs)
                for task in self.tasks[job]
                for decision_var, _, cost in assignment_task[(job, task)]
            )
        )
        # for job in range(self.n_jobs):
        #     for task in self.tasks[job]:
        #         # for machine in task_machines[task]:
        #         # print(
        #         #     f"Job {job} assigned to machine {machine} to perform task {task} has {assignment[(job, task, machine)].solution_value()}    "
        #         # )
        #         print(f"Task {job,task} ")
        #         for i, mc, _ in assignment_task[(job, task)]:
        #             # if i.solution_value() == 1:
        #             print(i)
        #         print()

        status = self.model.Solve()
        if status == pywraplp.Solver.OPTIMAL:
            print("Solution:")
            print("Objective value = ", self.model.Objective().Value())
            for job in range(self.n_jobs):
                for task in self.tasks[job]:
                    print(
                        "Job %d task %d starts at %f and ends at %f"
                        % (
                            job,
                            task,
                            self.start_vars[job][task].solution_value(),
                            self.end_vars[job][task].solution_value(),
                        )
                    )
        # t = 0
        for job in range(self.n_jobs):
            for task in self.tasks[job]:
                # for machine in task_machines[task]:
                # print(
                #     f"Job {job} assigned to machine {machine} to perform task {task} has {assignment[(job, task, machine)].solution_value()}    "
                # )
                print(f"Task {job,task} ")
                for i, _, _ in assignment_task[(job, task)]:
                    if i.solution_value() == 1:
                        print(i)
                print()
        # print(t)
        # t = 0
        # for ass in assignment:
        #     if assignment[ass].solution_value() > 0:
        #         print(ass, assignment[ass].solution_value())
        #         t += assignment[ass].solution_value()

    def solve(self):
        self.addConstraints()

In [46]:
from collections import defaultdict
tasks = [[1, 17, 2, 3], [1, 26, 3, 7, 5, 6], [1, 24, 3]]
orders = [
    [(1, 17), (17, 2), (2, 3)],
    [(1, 26), (26, 3), (3, 7), (7, 5)],
    [(1, 24), (24, 3)],
]
import pickle

with open("../datasets/capacities.pickle", "rb") as f:
    capacities = pickle.load(f)

In [47]:
model = LPJS(tasks, orders, capacities)
model.solve()

defaultdict(<class 'dict'>, {1: {25: (2.16, 1.76), 26: (4.03, 5.15)}, 2: {4: (4.34, 1.6)}, 3: {5: (0.28, 1.83)}, 5: {8: (2.76, 6.22), 9: (7.51, 5.6)}, 6: {3: (2.85, 5.1)}, 7: {22: (5.0, 4.29), 27: (6.59, 0.77), 28: (3.29, 9.94)}, 17: {1: (3.64, 7.35), 2: (3.98, 2.22), 6: (9.7, 4.35), 11: (3.72, 3.08), 13: (3.45, 5.27)}, 24: {15: (5.45, 5.35), 16: (2.23, 8.07), 18: (9.16, 2.51)}, 26: {11: (7.33, 5.05), 12: (3.34, 0.19)}})
Solution:
Objective value =  28.76
Job 0 task 1 starts at 0.000000 and ends at 4.030000
Job 0 task 17 starts at 4.030000 and ends at 13.730000
Job 0 task 2 starts at 13.730000 and ends at 18.070000
Job 0 task 3 starts at 18.070000 and ends at 18.350000
Job 1 task 1 starts at 0.000000 and ends at 4.030000
Job 1 task 26 starts at 4.030000 and ends at 11.360000
Job 1 task 3 starts at 11.360000 and ends at 11.640000
Job 1 task 7 starts at 11.640000 and ends at 18.230000
Job 1 task 5 starts at 18.230000 and ends at 100000000000000000000.000000
Job 1 task 6 starts at 0.00000

In [48]:
1.76 + 2.22 + 1.6 + 1.83 + 1.76 + 0.19 + 1.83 + 0.77 + 5.6 + 5.1 + 1.76 + 2.51 + 1.83

28.759999999999998